In [1]:
import pathlib
from xclim import subset
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob

In [2]:
# Path to files
path = "/home/mlopez/EXEC/Tg_annual_11_models/"
json = pathlib.Path("/home/mlopez/EXEC/Shapefiles/territoire_guide.shp")

In [3]:
files = list(glob.glob(os.path.join(path,'*.*')))
files

['/home/mlopez/EXEC/Tg_annual_11_models/ACCESS1-3_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/BNU-ESM_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/CanESM2_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/CMCC-CMS_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/GFDL-ESM2M_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/HadGEM2-CC_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/INM-CM4_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/IPSL-CM5A-LR_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/MPI-ESM-LR_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/NorESM1-M_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/NRCAN_obs_tg_mean_annual.nc']

In [4]:
ds1 = xr.open_dataset(files[0])

In [5]:
ds1

<xarray.Dataset>
Dimensions:  (lat: 320, lon: 416, time: 151)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lat      (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon      (lon) float32 -89.04521 -88.96188 -88.87855 ... -54.54659 -54.46326
Data variables:
    tg_mean  (time, lat, lon) float32 ...
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [6]:
import itertools

In [7]:
df = pd.DataFrame()

In [14]:
datasets = [xr.open_dataset(f) for f in files]
latlon_df_0 = pd.concat(pd.DataFrame(itertools.product(ds.lat.values, ds.lon.values), columns=["lat","lon"]) for ds in datasets).drop_duplicates(["lat","lon"])
latlon_df = gpd.GeoDataFrame(latlon_df_0)
latlon_df["geometry"] = [Point(lon, lat) for (lat,lon) in zip(latlon_df.lat, latlon_df.lon)]

In [16]:
latlon_df

,lat,lon,geometry
0,66.623306,-89.045212,POINT (-89.04521 66.62331)
1,66.623306,-88.961884,POINT (-88.96188 66.62331)
2,66.623306,-88.878548,POINT (-88.87855 66.62331)
3,66.623306,-88.795219,POINT (-88.79522 66.62331)
4,66.623306,-88.711891,POINT (-88.71189 66.62331)
...,...,...,...
544675,41.041668,-52.375004,POINT (-52.37500 41.04167)
544676,41.041668,-52.291672,POINT (-52.29167 41.04167)
544677,41.041668,-52.208336,POINT (-52.20834 41.04167)
544678,41.041668,-52.125004,POINT (-52.12500 41.04167)


In [17]:
shape = gpd.read_file(json)
#Reading Shapefiles in right coordinate system
shape = shape.to_crs({'init': 'epsg:4326'})
shape.head()

,TER_GUIDE,geometry
0,1a,"MULTIPOLYGON (((-72.70926 46.28173, -72.69524 ..."
1,2a,"POLYGON ((-76.08748 46.47438, -76.08807 46.475..."
2,2b,"MULTIPOLYGON (((-70.81729 47.01784, -70.81045 ..."
3,2c,"POLYGON ((-72.38130 45.65191, -72.38112 45.652..."
4,3ab,"POLYGON ((-79.06541 47.29977, -79.06474 47.299..."


In [21]:
%time latlon_res = gpd.sjoin(latlon_df, shape, op="within")

CPU times: user 22.8 s, sys: 2.04 s, total: 24.8 s
Wall time: 1.65 s


In [22]:
latlon_res

,lat,lon,geometry,index_right,TER_GUIDE
68896,52.873859,-67.712730,POINT (-67.71273 52.87386),24,6opqr
68897,52.873859,-67.629402,POINT (-67.62940 52.87386),24,6opqr
68898,52.873859,-67.546066,POINT (-67.54607 52.87386),24,6opqr
69312,52.790531,-67.712730,POINT (-67.71273 52.79053),24,6opqr
69313,52.790531,-67.629402,POINT (-67.62940 52.79053),24,6opqr
...,...,...,...,...,...
107945,45.040840,-72.295883,POINT (-72.29588 45.04084),3,2c
107946,45.040840,-72.212547,POINT (-72.21255 45.04084),3,2c
107947,45.040840,-72.129219,POINT (-72.12922 45.04084),3,2c
107948,45.040840,-72.045891,POINT (-72.04589 45.04084),3,2c


TODO:

1. Load a shapefile
2. Do the sjoin with that file
3. Save the result to a file. OR turn the above into a script, since it's perfectly fine to recompute each time you need it.